In [135]:
!pip install pypulseq==1.3.1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null

In [136]:
!wget https://github.com/MRsources/FLASHzero/archive/refs/heads/main.zip -O FLASHzero.zip &> /dev/null
!unzip FLASHzero.zip &> /dev/null
!mv FLASHzero-main/MRM2024 . &> /dev/null
!rm -rf FLASHzero.zip FLASHzero-main &> /dev/null

# Imports and definitions

In [160]:
import numpy as np
# newer numpy versions don't contain this, but pypulseq still relies on it
np.int = int
np.float = float
np.complex = complex

import MRzeroCore as mr0
import pypulseq as pp
import torch
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

Nread = 128
Nphase = 128

In [169]:
#@title define simulate function
def simulate_brain_2D(seq_fn,pixel=False,LL=False,verbose=False):
    # %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above
    sz = [128, 128]
    # (i) load a phantom object from file
    obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')

    obj_p = obj_p.interpolate(sz[0], sz[1], 1)
    obj_p.B1[:] *= 0.77 # Needed due to scanner B1
    if pixel:
      obj_test = obj_p.build()
      obj_p.T1[64,64] = torch.median(obj_test.T1)
      obj_p.T2[64,64] = torch.median(obj_test.T2)
      obj_p.B1[0,64,64] = torch.median(obj_test.B1)
      obj_p.B0[64,64] = torch.median(obj_test.B0)
      obj_p.D[64,64] = torch.median(obj_test.D)
      obj_p.T2dash[64,64] = 30*1e-3
      obj_p.PD[:] = 0
      obj_p.PD[64,64] = 1
    mask = [obj_p.PD>0]
    if verbose:
      obj_p.plot()
    # Convert Phantom into simulation data
    obj_p = obj_p.build()


    # %% S5:. SIMULATE  the external.seq file and add acquired signal to ADC plot
    # Read in the sequence
    seq0 = mr0.Sequence.import_file(seq_fn)
    # Simulate the sequence
    if pixel:
      graph = mr0.compute_graph(seq0, obj_p, 10000, 1e-6)
    else:
      graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
    if LL:
      signal = mr0.execute_graph(graph, seq0, obj_p, min_emitted_signal=1, min_latent_signal=1, print_progress=False)
    else:
      signal = mr0.execute_graph(graph, seq0, obj_p, print_progress=False)

    return signal, mask

In [162]:
#@title define Recon
def recon(signal,ktraj,Nread,Nphase):
    # %% S6: MR IMAGE RECON of signal ::: #####################################
    kspace_adc = torch.reshape((signal[:Nphase*Nread]), (Nphase, Nread)).clone().t()
    permvec=np.argsort(ktraj)
    kspace=kspace_adc[:,permvec]

    spectrum = torch.fft.fftshift(kspace)  # fftshift
    space = torch.fft.fft2(spectrum)  # FFT
    space = torch.fft.ifftshift(space)# fftshift
    return kspace, space

In [199]:
#@title define Plotting
def plot_img(reco,signal_unenc,reco_com,signal_com_unenc,reco_LL,signal_LL_unenc,label,label_com,mask):
      plt.subplot(231)
      plt.title('FFT-mag. ' + label)
      plt.imshow(np.abs(reco.numpy().T), origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(234)
      plt.title('FFT-phase ' + label)
      plt.imshow(np.angle(reco.numpy().T), vmin=-np.pi, vmax=np.pi, origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(232)
      plt.title('FFT-mag. ' + label_com)
      plt.imshow(np.abs(reco_com.numpy().T), origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(235)
      plt.title('FFT-phase ' + label_com)
      plt.imshow(np.angle(reco_com.numpy().T), vmin=-np.pi, vmax=np.pi, origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(233)
      plt.title('FFT-phase ' + "Look-Locker")
      plt.imshow(np.abs(reco_LL.numpy().T), origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(236)
      plt.title('FFT-phase ' + "Look-Locker")
      plt.imshow(np.angle(reco_LL.numpy().T), vmin=-np.pi, vmax=np.pi, origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.show()

      fig = plt.figure(figsize=(10, 10))
      plt.subplot(221)
      plt.title('FFT-mag. ' + label)
      plt.imshow(np.abs((reco*mask).numpy().T)-np.abs((reco_LL*mask).numpy().T), vmin=-75, vmax=75, origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(223)
      plt.title('FFT-phase ' + label)
      plt.imshow(np.angle(reco.numpy().T)-np.angle(reco_LL.numpy().T), vmin=-0.1, vmax=0.1, origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(222)
      plt.title('FFT-mag. ' + label_com)
      plt.imshow(np.abs((reco_com*mask).numpy().T)-np.abs((reco_LL*mask).numpy().T), vmin=-75, vmax=75, origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.subplot(224)
      plt.title('FFT-phase ' + label_com)
      plt.imshow(np.angle((reco_com*mask).numpy().T)-np.angle((reco_LL*mask).numpy().T), vmin=-0.1, vmax=0.1, origin="lower"); plt.axis('off')
      plt.colorbar()

      plt.show()

      plt.subplot(121)
      plt.plot(np.abs(signal_unenc[Nread//2:Nread*Nphase:Nread].numpy()),label=label)
      plt.plot(np.abs(signal_com_unenc[Nread//2:Nread*Nphase:Nread].numpy()),label=label_com)
      plt.plot(np.abs(signal_LL_unenc[Nread//2:Nread*Nphase:Nread].numpy()),label='Look-Locker')
      plt.legend()

      plt.subplot(122)
      plt.plot(np.angle(signal_unenc[Nread//2:Nread*Nphase:Nread].numpy()),label=label)
      plt.plot(np.angle(signal_com_unenc[Nread//2:Nread*Nphase:Nread].numpy()),label=label_com)
      plt.plot(np.angle(signal_LL_unenc[Nread//2:Nread*Nphase:Nread].numpy()),label='Look-Locker')
      plt.ylim(-np.pi/2,np.pi/4)
      plt.legend()
      plt.show()

# Simulation

In [202]:
#@title Run Simulation
# Definiere die beiden Pfade
default_directory_path = 'MRM2024/seq-file/encoded/resolution_128'
alternative_directory_path = 'MRM2024/seq-file/unencoded'

# Funktion zum Laden der Dateien aus dem default Verzeichnis
def load_files_from_default_directory():
    files = os.listdir(default_directory_path)
    dropdown.options = files
    dropdown_com.options = files
    start_button.disabled = False  # Start-Button aktivieren

# Checkbox erstellen
P_PE_rewinder_flag = widgets.Checkbox(
    value=True,
    description='P_PE_rewinder_flag'
)

# Dropdown-Menü erstellen (leer, wird später gefüllt)
dropdown = widgets.Dropdown(
    options=[],
    description='File 1:'
)

# Dropdown-Menü erstellen (leer, wird später gefüllt)
dropdown_com = widgets.Dropdown(
    options=[],
    description='File 2:'
)

# Start-Button erstellen
start_button = widgets.Button(
    description='Start Simulation',
    disabled=True,  # Start-Button ist anfangs deaktiviert
    button_style='info',  # Ändere den Button-Stil zu 'info'
    tooltip='Click to start simulation',
    icon='play'  # (FontAwesome names without the `fa-` prefix)
)

# Ausführungsfenster erstellen
output = widgets.Output()

# Funktion zum Aktualisieren des Dropdown-Menüs basierend auf der Checkbox
def update_dropdown(change):
    if change['new']:
        # Wenn Checkbox aktiviert ist, lade Dateien aus dem default Verzeichnis
        load_files_from_default_directory()
    else:
        # Wenn Checkbox deaktiviert ist, lade Dateien aus dem alternativen Verzeichnis
        if os.path.exists(alternative_directory_path):
            files = os.listdir(alternative_directory_path)
            dropdown.options = files
            dropdown_com.options = files
            start_button.disabled = False  # Start-Button aktivieren
        else:
            dropdown.options = []
            start_button.disabled = True  # Start-Button deaktivieren

# Standardmäßig Dateien aus dem default Verzeichnis laden
load_files_from_default_directory()

# Verknüpfe die Checkbox mit der Update-Funktion
P_PE_rewinder_flag.observe(update_dropdown, names='value')

# Funktion, die aufgerufen wird, wenn der Start-Button gedrückt wird
def start_simulation(button):
    with output:
        print("Simulation started for file (encoded):", dropdown.value)
        seq_fn = os.path.join(default_directory_path,dropdown.value)
        signal,_ = simulate_brain_2D(seq_fn,pixel=False,verbose=False)
        ## centric reordering
        phenc = np.arange(-Nphase // 2, Nphase // 2, 1)
        permvec =np.arange(0, Nphase, 1)
        ktraj = sorted(np.arange(len(phenc)), key=lambda x: abs(len(phenc) // 2 - x))
        kspace, reco = recon(signal,ktraj,Nread,Nphase)
        print("Simulation started for file (unencoded):", dropdown.value)
        seq_fn = os.path.join(alternative_directory_path,dropdown.value)
        signal_unenc,_ = simulate_brain_2D(seq_fn,pixel=True,verbose=False)

        print("Simulation started for file (encoded):", dropdown_com.value)
        seq_fn = os.path.join(default_directory_path,dropdown_com.value)
        signal_com,_ = simulate_brain_2D(seq_fn,pixel=False,verbose=False)
        ## centric reordering
        phenc = np.arange(-Nphase // 2, Nphase // 2, 1)
        permvec =np.arange(0, Nphase, 1)
        ktraj = sorted(np.arange(len(phenc)), key=lambda x: abs(len(phenc) // 2 - x))
        kspace_com, reco_com = recon(signal_com,ktraj,Nread,Nphase)
        print("Simulation started for file (unencoded):", dropdown_com.value)
        seq_fn = os.path.join(alternative_directory_path,dropdown_com.value)
        signal_com_unenc,_ = simulate_brain_2D(seq_fn,pixel=True,verbose=False)

        print("Simulation started for Look-Locker (encoded)!")
        file = "rfInc_117_TR_6.seq"
        seq_fn = os.path.join(default_directory_path,file)
        signal_LL,mask = simulate_brain_2D(seq_fn,pixel=False,LL=True,verbose=False)
        ## centric reordering
        phenc = np.arange(-Nphase // 2, Nphase // 2, 1)
        permvec =np.arange(0, Nphase, 1)
        ktraj = sorted(np.arange(len(phenc)), key=lambda x: abs(len(phenc) // 2 - x))
        kspace_LL, reco_LL = recon(signal_LL,ktraj,Nread,Nphase)
        print("Simulation started for Look-Locker (unencoded)!")
        seq_fn = os.path.join(alternative_directory_path,file)
        signal_LL_unenc,_ = simulate_brain_2D(seq_fn,pixel=True,LL=True,verbose=False)
        plot_img(reco,signal_unenc,reco_com,signal_com_unenc,reco_LL,signal_LL_unenc,dropdown.value,dropdown_com.value,mask[0][...,0])

# Verknüpfe den Start-Button mit der Start-Simulations-Funktion
start_button.on_click(start_simulation)

# Stiländerung für die Widgets
P_PE_rewinder_flag.layout.width = 'auto'
dropdown.layout.width = 'auto'
dropdown_com.layout.width = 'auto'
start_button.layout.width = 'auto'

# Widgets anzeigen
display(widgets.VBox([dropdown, dropdown_com, start_button, output]))